In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
data = pd.read_parquet('./features_ready_220507.parquet.gzip')
data = data.drop(['Open', 'High', 'Low', 'Close', 'Volume','AdjustmentFactor','ExpectedDividend','SupervisionFlag','CumFactor',
                       'SMA50','SMA10','k','EMA12','EMA26','SMA14UP','SMA14DOWN','Lower_line','Upper_line',
                       'SD10','MACD','EMA9','tot_volume','ind_volume'], axis=1)
print(data.columns)

Index(['Date', 'SecuritiesCode', 'Target', 'Mo', 'Tu', 'We', 'Th', 'Fr',
       'DividendRatio', 'SMA50_Close_diff', 'SMA10_Close_diff', 'Volume_share',
       'UL_Close-diff', 'LL_Close-diff', 'RSI', 'Stochastic', 'EMA_diff',
       'BeforeDiv', 'AfterDiv', 'Day_number', 'Close_lag_1', 'Gap',
       'Open_lag_1', 'Close_lag_2', 'Open_lag_2', 'Close_lag_3', 'Open_lag_3',
       'Candle', 'Target_lag_2', 'Target_lag_3', 'Target_lag_4'],
      dtype='object')


In [3]:
print(data[data['Date'] == '2021-12-06'].shape, data[data['Date'] == '2021-12-06'])

(2000, 31)               Date  SecuritiesCode  Target  Mo  Tu  We  Th  Fr  DividendRatio  \
index                                                                           
2332531 2021-12-06            1301     0.0   1   0   0   0   0            0.0   
2332532 2021-12-06            1332     0.0   1   0   0   0   0            0.0   
2332533 2021-12-06            1333     0.0   1   0   0   0   0            0.0   
2332534 2021-12-06            1375     0.0   1   0   0   0   0            0.0   
2332535 2021-12-06            1376     0.0   1   0   0   0   0            0.0   
...            ...             ...     ...  ..  ..  ..  ..  ..            ...   
2334526 2021-12-06            9990     0.0   1   0   0   0   0            0.0   
2334527 2021-12-06            9991     0.0   1   0   0   0   0            0.0   
2334528 2021-12-06            9993     0.0   1   0   0   0   0            0.0   
2334529 2021-12-06            9994     0.0   1   0   0   0   0            0.0   
2334530 2021-12-0

In [4]:
X_train = data[-data['Date'].isin(['2021-12-06', '2021-12-07'])]
X_train.fillna(0, inplace=True)
y_train = X_train['Target']
X_train = X_train.drop(['Date', 'SecuritiesCode', 'Target'], axis=1)

X_test = data[data['Date'].isin(['2021-12-06'])]
X_test.fillna(0, inplace = True)
X_sub = X_test[['Date', 'SecuritiesCode']].reset_index()
X_test = X_test.drop(['Date', 'SecuritiesCode', 'Target'], axis=1)


/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_7659/2929529245.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(0, inplace=True)
/var/folders/wp/0d8l7yys1t38lg64tnywmtqw0000gn/T/ipykernel_7659/2929529245.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace = True)


In [5]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.005798878123100426

In [6]:
y = pd.DataFrame(reg.predict(X_test), columns = ['Predicted_Target'])

In [7]:
result = pd.concat([X_sub, y], axis=1)
result['Predicted_Rank'] = result['Predicted_Target'].rank()

In [8]:
test_list = pd.read_csv('files/supplemental_files/stock_prices.csv', index_col=0, parse_dates=[1])
test_list = test_list[test_list['Date']=='2021-12-06'][['SecuritiesCode', 'Target']]
test_list['Real_Rank'] = test_list['Target'].rank()
test_list.rename(columns ={'Target':'True_value'}, inplace = True)

In [9]:
score_calc = result.merge(test_list, how='left', on='SecuritiesCode')
score_calc

,index,Date,SecuritiesCode,Predicted_Target,Predicted_Rank,True_value,Real_Rank
0,2332531,2021-12-06,1301,0.000458,1226.0,-0.003263,606.0
1,2332532,2021-12-06,1332,0.000505,1316.0,-0.008993,330.0
2,2332533,2021-12-06,1333,0.000224,794.0,-0.009963,288.0
3,2332534,2021-12-06,1375,0.000750,1654.0,-0.015032,156.0
4,2332535,2021-12-06,1376,0.000837,1720.0,0.002867,1022.0
...,...,...,...,...,...,...,...
1995,2334526,2021-12-06,9990,0.000031,462.0,0.009346,1381.0
1996,2334527,2021-12-06,9991,0.000680,1572.0,0.001242,904.0
1997,2334528,2021-12-06,9993,0.000661,1552.0,0.011728,1474.0
1998,2334529,2021-12-06,9994,0.000436,1181.0,-0.001230,744.0


In [10]:
def top_calc(row):
    pred_rank = row['Predicted_Rank']
    if pred_rank < 200:
        return (2- pred_rank / 200) * row['Predicted_Target'] / 1.5
def bot_calc(row):
    pred_rank = row['Predicted_Rank']
    if pred_rank >=1800:
        return (2- np.abs(pred_rank - 1999) / 200) * row['Predicted_Target'] / 1.5

score_calc['Top_result'] = score_calc.apply(top_calc, axis=1)
score_calc['Bot_result'] = score_calc.apply(bot_calc, axis=1)

sharpe = score_calc['Top_result'].sum() - score_calc['Bot_result'].sum()

In [11]:
sharpe

-0.4833767916782674